# Загрузка базы подготовка базы к обучению AutoML


In [5]:
import matplotlib.pyplot as plt # matplotlib для графиков
import seaborn as sns # seaborn для графиков
import numpy as np    # Нампай для массивов
import pandas as pd   # Пандас для данных
import re             # Регулярные выражения
from tensorflow.keras import utils # Для приведения к категориальным данным
from google.colab import files     # Загрузка файлов

In [6]:
# Для загрузки данных используем drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
# Загружаем базу
df = pd.read_csv('/content/dataset1.csv')
df

<ipython-input-7-871d2d653d97>:2: DtypeWarning: Columns (16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/dataset1.csv')


,id_ответа,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,...,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Поставьте галочку, если уверены на 100%",Если 75%,Если 50%,"Альтернатива, если 50% - обязательно",Комментарий
0,1,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,до 42 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0
1,2,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,от 43 до 60 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0
2,3,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,более 61 дня,нет мутаций,Не исследовался,Результативность лечения,Наблюдение,False,True,False,Дурвалумаб,Нет доказательной базы для назначения после пе...
3,4,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,до 42 дней,не исследовались,Не исследовался,Результативность лечения,Дурвалумаб,False,True,False,0,Возможно есть мутации и эффективность Дурвалум...
4,5,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,от 43 до 60 дней,не исследовались,Не исследовался,Результативность лечения,Дурвалумаб,False,True,False,0,Возможно есть мутации и эффективность Дурвалум...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40954,40955,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,от 43 до 60 дней,не исследовались,Менее 1%,Результативность лечения,Наблюдение,False,True,False,Дурвалумаб,Возможна низкая эффективность Дурвалумаба при ...
40955,40956,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,более 61 дня,не исследовались,Менее 1%,Результативность лечения,Наблюдение,False,True,False,0,Нет доказательной базы для назначения после пе...
40956,40957,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,до 42 дней,EGFR редкий вариант,Менее 1%,Результативность лечения,Наблюдение,True,False,False,0,"Возможна низкая эффективность Дурвалумаба, Оси..."
40957,40958,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,от 43 до 60 дней,EGFR редкий вариант,Менее 1%,Результативность лечения,Наблюдение,True,False,False,0,"Возможна низкая эффективность Дурвалумаба, Оси..."


# Устанавливаем библиотеку для работы с autokeras



In [8]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.5/129.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.1/590.1 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 42.9 MB/s eta 0:00:00


In [9]:
import pandas as pd
from IPython.display import display
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import csv
import autokeras as ak
from sklearn.preprocessing import LabelEncoder

Using TensorFlow backend


In [10]:
# Запишем датасет в переменную и выведем на экран
df = pd.read_csv('dataset1.csv', quoting=csv.QUOTE_NONE)
display(df)

,id_ответа,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,...,PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"""Поставьте галочку","если уверены на 100%""",Если 75%,Если 50%,"""Альтернатива","если 50% - обязательно""",Комментарий
0,1.0,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0,NaN,NaN
1,2.0,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,Не исследовался,Результативность лечения,Дурвалумаб,True,False,False,0,0,NaN,NaN
2,3.0,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,Не исследовался,Результативность лечения,Наблюдение,False,True,False,Дурвалумаб,Нет доказательной базы для назначения после пе...,NaN,NaN
3,4.0,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,Не исследовался,Результативность лечения,Дурвалумаб,False,True,False,0,Возможно есть мутации и эффективность Дурвалум...,NaN,NaN
4,5.0,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,...,Не исследовался,Результативность лечения,Дурвалумаб,False,True,False,0,Возможно есть мутации и эффективность Дурвалум...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42470,40955.0,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,Менее 1%,Результативность лечения,Наблюдение,False,True,False,Дурвалумаб,"""Возможна низкая эффективность Дурвалумаба при...","возможно есть активирующие мутации""",NaN
42471,40956.0,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,Менее 1%,Результативность лечения,Наблюдение,False,True,False,0,Нет доказательной базы для назначения после пе...,NaN,NaN
42472,40957.0,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,Менее 1%,Результативность лечения,Наблюдение,True,False,False,0,"""Возможна низкая эффективность Дурвалумаба","Осимертиниб без доказательной базы""",NaN
42473,40958.0,Европейская,Мужской,>70,Курение в прошлом (бросил более 1 месяца до 1...,0-1,нет,да,нет,да,...,Менее 1%,Результативность лечения,Наблюдение,True,False,False,0,"""Возможна низкая эффективность Дурвалумаба","Осимертиниб без доказательной базы""",NaN


In [11]:
# Выведем информацию по датасету
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42475 entries, 0 to 42474
Data columns (total 23 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   id_ответа                                                 40959 non-null  float64
 1   Раса                                                      42475 non-null  object 
 2   Пол                                                       40959 non-null  object 
 3   Возраст                                                   40958 non-null  object 
 4   Статус курения                                            40958 non-null  object 
 5   ECOG                                                      40958 non-null  object 
 6   Есть опухолевая нагрузка? (симптомная опухоль)            40958 non-null  object 
 7   Ко-мутации KRAS                                           40958 non-null  object 
 8   Ко-мутации p53. 

In [12]:
# Создание экземпляра LabelEncoder
label_encoder = LabelEncoder()

# Применение LabelEncoder ко всем столбцам
for col in df.columns:
    if df[col].dtype == 'object':  # Проверка, что столбец имеет тип 'object'
        df[col] = label_encoder.fit_transform(df[col].astype(str))

In [13]:
# Перепроверим информацию по датасету по изменению типа данных
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42475 entries, 0 to 42474
Data columns (total 23 columns):
 #   Column                                                    Non-Null Count  Dtype  
---  ------                                                    --------------  -----  
 0   id_ответа                                                 40959 non-null  float64
 1   Раса                                                      42475 non-null  int64  
 2   Пол                                                       42475 non-null  int64  
 3   Возраст                                                   42475 non-null  int64  
 4   Статус курения                                            42475 non-null  int64  
 5   ECOG                                                      42475 non-null  int64  
 6   Есть опухолевая нагрузка? (симптомная опухоль)            42475 non-null  int64  
 7   Ко-мутации KRAS                                           42475 non-null  int64  
 8   Ко-мутации p53. 

In [14]:
# Отделим входные переменные
x = df[df.columns[2:15]]
x

,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,Ко-мутации KEAP1,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию
0,2,1,0,0,0,0,0,0,0,2,5,2,0
1,2,1,0,0,0,0,0,0,0,3,5,2,0
2,2,1,0,0,0,0,0,0,0,1,5,2,0
3,2,1,0,0,0,0,0,0,0,2,4,2,0
4,2,1,0,0,0,0,0,0,0,3,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42470,2,0,2,0,1,0,1,0,0,3,4,1,0
42471,2,0,2,0,1,0,1,0,0,1,4,1,0
42472,2,0,2,0,1,0,1,0,0,2,3,1,0
42473,2,0,2,0,1,0,1,0,0,3,3,1,0


In [15]:
# Проверочные переменные
y = df[df.columns[16:21]]
y

,"""Поставьте галочку","если уверены на 100%""",Если 75%,Если 50%,"""Альтернатива"
0,1,0,0,0,3
1,1,0,0,0,3
2,0,1,0,3,8
3,0,1,0,0,7
4,0,1,0,0,7
...,...,...,...,...,...
42470,0,1,0,3,1
42471,0,1,0,0,8
42472,1,0,0,0,0
42473,1,0,0,0,0


In [16]:
# Разбиение наборов на общую и проверочную выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, shuffle=True, random_state = 15)

In [17]:
# Вывод размеров обучающей и проверочной выборок
print(f'Размер обучающей выборки: {x_train.shape[0]}')
print(f'Размер проверочной выборки: {x_test.shape[0]}')

Размер обучающей выборки: 36103
Размер проверочной выборки: 6372


## **Эксперементы**

# Эксперимент 1. Tuner по умолчанию, 10 запусков.

In [19]:
# Инициализируйте классификатор структурированных данных.
reg = ak.StructuredDataRegressor(
    overwrite=True,
    max_trials=10,
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment1',
)

# поиск лучшей модели
reg.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 10 Complete [00h 01m 38s]
val_loss: 1.67372465133667

Best val_loss So Far: 0.0005500743282027543
Total elapsed time: 00h 16m 15s
Epoch 1/10
1129/1129 [==============================] - 8s 6ms/step - loss: 1.2180 - mean_absolute_error: 0.6958 - val_loss: 0.4409 - val_mean_absolute_error: 0.4191
Epoch 2/10
1129/1129 [==============================] - 4s 4ms/step - loss: 0.2658 - mean_absolute_error: 0.3180 - val_loss: 0.1127 - val_mean_absolute_error: 0.2135
Epoch 3/10
1129/1129 [==============================] - 5s 5ms/step - loss: 0.0515 - mean_absolute_error: 0.1486 - val_loss: 0.0189 - val_mean_absolute_error: 0.0991
Epoch 4/10
1129/1129 [==============================] - 6s 5ms/step - loss: 0.0102 - mean_absolute_error: 0.0728 - val_loss: 0.0044 - val_mean_absolute_error: 0.0494
Epoch 5/10
1129/1129 [==============================] - 6s 5ms/step - loss: 0.0029 - mean_absolute_error: 0.0401 - val_loss: 0.0021 - val_mean_absolute_error: 0.0347
Epoch 6/10
1129/1129 [============

In [20]:
best_model = reg.export_model()

In [21]:
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 multi_category_encoding (M  (None, 13)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 13)                27        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                448       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

# Эксперимент №2. Tuner "hyperband", 5 запусков.

In [22]:
reg2 = ak.StructuredDataRegressor(
    overwrite=True,
    max_trials=5,
    tuner="hyperband",
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment2'
)

# поиск лучшей модели
reg2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 50s]
val_loss: 0.784576952457428

Best val_loss So Far: 0.44628024101257324
Total elapsed time: 00h 05m 10s
Epoch 1/10
1129/1129 [==============================] - 12s 10ms/step - loss: 1.5035 - mean_absolute_error: 0.8340 - val_loss: 0.6192 - val_mean_absolute_error: 0.5638
Epoch 2/10
1129/1129 [==============================] - 12s 11ms/step - loss: 1.1505 - mean_absolute_error: 0.7299 - val_loss: 0.4648 - val_mean_absolute_error: 0.4931
Epoch 3/10
1129/1129 [==============================] - 11s 10ms/step - loss: 1.0682 - mean_absolute_error: 0.7032 - val_loss: 0.4179 - val_mean_absolute_error: 0.4807
Epoch 4/10
1129/1129 [==============================] - 11s 10ms/step - loss: 1.0272 - mean_absolute_error: 0.6893 - val_loss: 0.3423 - val_mean_absolute_error: 0.4312
Epoch 5/10
1129/1129 [==============================] - 10s 8ms/step - loss: 0.9931 - mean_absolute_error: 0.6801 - val_loss: 0.3103 - val_mean_absolute_error: 0.4158
Epoch 6/10
1129/1129 [=====

In [23]:
best_model2 = reg2.export_model()

In [24]:
best_model2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 multi_category_encoding (M  (None, 13)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 13)                27        
 on)                                                             
                                                                 
 dense (Dense)               (None, 1024)              14336     
                                                                 
 batch_normalization (Batch  (None, 1024)              4096      
 Normalization)                                                  
                                                             

# Эксперимент №3. AutoModel. Tuner "greedy", 5 запусков.

In [25]:
reg3 = ak.AutoModel(
    inputs=ak.StructuredDataInput(),
    outputs=ak.RegressionHead(),
    max_trials=5,
    tuner="greedy",
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment3',
    overwrite=True)
reg3.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# оценка модели
results = reg3.evaluate(x_test, y_test)
print(results)


Trial 5 Complete [00h 01m 45s]
val_loss: 0.07388108968734741

Best val_loss So Far: 0.07388108968734741
Total elapsed time: 00h 08m 20s
Epoch 1/10
1129/1129 [==============================] - 6s 5ms/step - loss: 0.9468 - mean_absolute_error: 0.6188 - val_loss: 0.2316 - val_mean_absolute_error: 0.3303
Epoch 2/10
1129/1129 [==============================] - 6s 5ms/step - loss: 0.4329 - mean_absolute_error: 0.4325 - val_loss: 0.1520 - val_mean_absolute_error: 0.2660
Epoch 3/10
1129/1129 [==============================] - 5s 4ms/step - loss: 0.3738 - mean_absolute_error: 0.4078 - val_loss: 0.1548 - val_mean_absolute_error: 0.2781
Epoch 4/10
1129/1129 [==============================] - 6s 5ms/step - loss: 0.3453 - mean_absolute_error: 0.3942 - val_loss: 0.1324 - val_mean_absolute_error: 0.2746
Epoch 5/10
1129/1129 [==============================] - 4s 4ms/step - loss: 0.3343 - mean_absolute_error: 0.3889 - val_loss: 0.1387 - val_mean_absolute_error: 0.2423
Epoch 6/10
1129/1129 [============

In [26]:
best_model3 = reg3.export_model()

In [27]:
best_model3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 multi_category_encoding (M  (None, 13)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 13)                27        
 on)                                                             
                                                                 
 dense (Dense)               (None, 64)                896       
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                                 
 dropout (Dropout)           (None, 64)                0     